# TUTORIAL PLEIOTROPY (PyViper)

# Step 1: Import Libraries

In [1]:

import numpy as np
import pandas as pd
from scipy.stats import norm
import scanpy as sc
import pyviper
import matplotlib.pyplot as plt 
from itertools import combinations

/mnt/beegfs/home/friva/miniconda3/envs/r_bioc320_env/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


# Step 2: Import gene expression matrix

In [2]:
# Set the HPC file path
gene_expr_path = "/home/friva/TCGA_DEG/DEG-BRCA.tsv"
# Read the data
gene_expr_signature = pd.read_csv(gene_expr_path, sep="\t", index_col=0)
# Convert to AnnData object
gene_expr_signature = sc.AnnData(gene_expr_signature)
# Inspect
gene_expr_signature

AnnData object with n_obs × n_vars = 1106 × 19938

# Step 3: Load the prunned ARACNe Network (GRN)

In [3]:

network_path = "/home/friva/TCGA_networks/net-prunned-BRCA_test.tsv"

network = pd.read_csv(network_path, delimiter="\t")

# Drop the first column if it's just an index
network = network.drop(network.columns[0], axis=1)

network_interactome = pyviper.Interactome('TCGA_BRCA', network)  # convert to class Interactome
network_interactome.size()  # show number of regulons in the Interactome
network_interactome.net_table.head()


,regulator,target,mor,likelihood
0,ABCG1,PLEKHF2,1.000000,0.986344
1,ABCG1,WWP1,0.999999,0.981491
2,ABCG1,SERPINB5,-0.999987,0.977465
3,ABCG1,HRCT1,-0.999980,0.970370
4,ABCG1,PLAGL1,-0.999981,0.970235


# Step 4: Filter the network to contain only the targets present in the gene expression signature

In [4]:
# keep targets present in expression
network_interactome.filter_targets(gene_expr_signature.var_names)
# prune to exactly 50 targets per TF (drop TFs with <50)
network_pruned = network_interactome.copy()
network_pruned.prune(max_targets=50, eliminate=True)

Removed 0 targets.
Removed 15308 targets.
Removed 412 regulators.


/mnt/beegfs/home/friva/miniconda3/envs/r_bioc320_env/lib/python3.10/site-packages/pyviper/interactome.py:594: FutureWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  pruned_df = sorted_df.groupby('regulator', group_keys=False).apply(lambda x: x.iloc[:max_targets])


In [5]:
# Save to TSV for R
# Save to TSV for R (no index, tab-separated)
network_pruned.net_table.to_csv("Interactome_prunedBRCA1.tsv", sep="\t", index=False)


# Step 5: Run PyViper with no Pleiotropy applied and save the NES scores output

In [6]:
ProtAct_aREA = pyviper.viper(gex_data=gene_expr_signature, # gene expression signature
                             interactome=network_pruned, # gene regulatory network
                             enrichment = "area",
                             output_as_anndata=False,
                             min_targets=0,
                             eset_filter=True,
                             njobs=1,
                             verbose=False)

In [7]:
ProtAct_aREA

,ABLIM3,ACTL6B,AFF3,AGO2,AGRN,AJUBA,ARID2,ARID5A,ASF1B,ASXL2,...,YEATS2,ZEB2,ZFYVE28,ZGPAT,ZMYND10,ZMYND15,ZNF267,ZNF385A,ZNF446,ZNF688
TCGA-AC-A8OP-01A,3.312995,0.138986,1.413288,-1.612328,3.334114,-0.849215,0.904308,2.418113,-3.328123,-0.200537,...,-2.832208,-0.706952,5.428454,5.537533,1.452248,1.033461,-2.721778,4.275638,4.272559,1.816097
TCGA-D8-A1XU-01A,4.402324,0.179526,1.666932,-2.186245,-2.240458,-0.768459,4.181095,-1.470134,-5.360287,2.623855,...,-3.227953,3.354212,-1.347843,-4.795663,-0.271248,-0.773999,2.810309,2.763715,-1.091221,-0.254045
TCGA-BH-A18L-01A,-1.004362,2.151375,3.103410,5.265637,-4.065179,-4.552784,1.567419,-4.918112,3.740328,4.462186,...,0.935340,-4.970821,-0.495176,-1.155153,1.028837,-5.408047,1.563194,-0.209267,-0.041467,-0.084053
TCGA-B6-A0IK-01A,-3.181720,0.048878,-2.066078,2.308023,1.347444,-2.042295,2.173005,-1.745898,3.310119,-1.272703,...,1.294032,-2.918173,-5.684632,-0.258629,-4.526424,-5.167581,-1.332390,-0.779405,-4.365264,-2.323720
TCGA-BH-A0B1-01A,0.155697,-0.108404,0.632171,2.215284,-3.398050,0.351922,3.168435,-3.730084,-1.739547,6.122243,...,0.080908,0.017950,-1.356516,-4.137085,1.327284,-5.181332,3.391932,1.053169,-1.686188,-1.370462
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
TCGA-GM-A2DN-01A,0.011731,0.472961,-1.289979,-2.624401,0.360927,0.668203,-1.216661,3.181600,-1.962677,-3.307842,...,-2.489465,3.038561,1.958210,-0.417439,3.264119,5.168600,3.658692,2.895154,1.794714,1.334522
TCGA-B6-A0I1-01A,-4.144918,0.594553,-4.380195,2.769082,-0.436839,-1.303278,-6.380695,2.066891,4.353570,-6.028299,...,5.279651,-4.069907,-0.656741,1.537583,-2.449949,2.816692,-0.222014,-3.031335,-1.741875,-0.764202
TCGA-EW-A1IW-01A,0.850925,-0.615537,1.412039,1.495621,-3.623744,0.270178,4.274262,-2.913677,1.543674,5.599647,...,0.550692,-0.334294,-3.508375,-6.054484,2.415661,-5.130964,0.562882,-0.494052,-2.238624,-2.200464
TCGA-BH-A42V-01A,4.441164,-0.460793,0.829335,-7.201144,6.484625,1.397921,-3.041894,6.647077,-7.835933,-6.195728,...,-5.248777,-1.433316,7.697809,7.824760,0.830612,4.134101,-7.093821,2.909948,8.023783,4.866827


In [8]:
NES_pre_py = ProtAct_aREA.T.copy()  # TFs x samples
NES_pre_py.index = NES_pre_py.index.astype(str)
NES_pre_py.columns = NES_pre_py.columns.astype(str)
NES_pre_py.to_csv("NES_pyVIPER_precorrectionBRCA1.csv")


# Step 6: Convert Intercatome to Regulon to mimic R behavior 

In [9]:
from typing import Dict

# optional progress bar
try:
    from tqdm import tqdm
except ImportError:
    def tqdm(x, **kwargs):
        return x
# Function to Convert Interactome to Regulon (TabletoRegulon)
def TableToInteractome(net_table: pd.DataFrame) -> Dict[str, Dict[str, pd.Series]]:
    # First block of TableToInteractome: initialize my_reg with a blank
    # structure for each regulator that appears in net_table.
    # 1) blank_reg in R -> we'll create a fresh copy per TF in Python
    #    (do NOT reuse the same object across TFs)
    def _fresh_blank():
        # make sure each TF gets a space for tfmode and likelihood
        return {"tfmode": [], "likelihood": [], "_targets": []}

    # Purpose first loop: just to initialize each regulator with a blank regulon
    # At the end of this, every TF has an empty tfmode and likelihood.
    my_reg: Dict[str, Dict[str, list]] = {}

    # loop over each row with progress bar
    for _, row in tqdm(net_table.iterrows(), total=len(net_table), desc="Initializing regulons"):
        regulator = row["regulator"]
        # Overwrite every time (like in R)
        my_reg[regulator] = _fresh_blank()

    # Purpose second loop: filling in the values from the net_table for each target and regulator
    for _, row in tqdm(net_table.iterrows(), total=len(net_table), desc="Filling regulons"):
        r = row["regulator"]
        t = row["target"]
        w = row["likelihood"]
        m = row["mor"]

        # In Python we collect targets, then set them as index (names) at the end
        if r not in my_reg:
            my_reg[r] = _fresh_blank()
        my_reg[r]["_targets"].append(str(t))
        my_reg[r]["likelihood"].append(float(w))
        my_reg[r]["tfmode"].append(float(m))

    # --- Finalize: convert lists to named Series (index = target), like R named vectors ---
    for r, reg in my_reg.items():
        targets = reg["_targets"]
        reg["likelihood"] = pd.Series(reg["likelihood"], index=targets, dtype=float)
        reg["tfmode"]     = pd.Series(reg["tfmode"],     index=targets, dtype=float)
        del reg["_targets"]  # cleanup helper list

    return my_reg


In [10]:
regulon = TableToInteractome(network_pruned.net_table)

Filling regulons: 100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████| 10400/10400 [00:00<00:00, 30587.32it/s]


# Step 7: ShadowRegulon_py 

In [11]:

import numpy as np
import pandas as pd
from scipy.stats import norm
from itertools import combinations
from typing import Dict, Optional

def ShadowRegulon_py(
    ss: pd.Series,
    nes: pd.Series,
    regul: Dict[str, Dict[str, pd.Series]],
    regulators: float = 0.05,
    shadow: float = 0.05,
    targets: int = 10,
    penalty: float = 2.0,
    method: str = "absolute"
) -> Optional[Dict[str, Dict[str, pd.Series]]]:
    # ---- method check (R: match.arg) ----
    method = str(method).lower()
    if method not in ("absolute", "adaptive"):
        raise ValueError("method must be 'absolute' or 'adaptive'")

    # ---- R-like coercions/names ----
    if not isinstance(ss, pd.Series):
        ss = pd.Series(ss)
    if not isinstance(nes, pd.Series):
        nes = pd.Series(nes)
    ss = ss.copy(); nes = nes.copy()
    ss.index = ss.index.astype(str)
    nes.index = nes.index.astype(str)

    # ---- two-sided p from NES under N(0,1) ----
    pval_all = pd.Series(2.0 * norm.sf(np.abs(nes.astype("float64"))), index=nes.index)

    # ---- choose candidate TFs: cutoff or top-N ----
    if regulators < 1:
        tfs = list(pval_all.index[pval_all.values < regulators])
    else:
        k = min(int(regulators), len(pval_all))
        order_idx = np.argsort(pval_all.values)[:k]
        tfs = list(pval_all.index[order_idx])

    # ---- drop composite names and require >=2 TFs ----
    tfs = [tf for tf in tfs if "--" not in tf]
    if len(tfs) < 2:
        return None

    # ---- unique(tfs) (order-preserving) ----
    unique_tfs = list(dict.fromkeys(tfs))

    # ---- per-tf1 list of per-tf2 one-sided p-values ----
    tmp: Dict[str, pd.Series] = {}

    for tf1 in unique_tfs:
        missing_tf1 = tf1 not in regul or 'tfmode' not in regul[tf1] or 'likelihood' not in regul[tf1]
        if not missing_tf1:
            tf1_tfmode = regul[tf1]['tfmode'].copy()
            tf1_like   = regul[tf1]['likelihood'].copy()
            tf1_tfmode.index = tf1_tfmode.index.astype(str)
            tf1_like.index   = tf1_like.index.astype(str)
        else:
            tf1_tfmode = pd.Series(dtype="float64")
            tf1_like   = pd.Series(dtype="float64")

        tf2_list = [x for x in tfs if x != tf1]

        # build overlap slices as a list (to allow duplicate tf2 labels, like R list)
        reg_overlap_list = []
        for tf2 in tf2_list:
            if missing_tf1 or tf2 not in regul or 'tfmode' not in regul[tf2]:
                reg_overlap_list.append((tf2, {
                    'tfmode': pd.Series(dtype="float64"),
                    'likelihood': pd.Series(dtype="float64")
                }))
                continue
            tf2_tfmode = regul[tf2]['tfmode'].copy()
            tf2_tfmode.index = tf2_tfmode.index.astype(str)
            pos_mask = tf1_tfmode.index.isin(tf2_tfmode.index)
            reg_overlap_list.append((
                tf2,
                {
                    'tfmode': tf1_tfmode.loc[pos_mask],
                    'likelihood': tf1_like.loc[pos_mask]
                }
            ))

        # s1/s2 transforms on genes regulated by tf1
        tf1_genes = tf1_tfmode.index
        ss_pos_mask = ss.index.isin(tf1_genes)
        ss_slice = ss.loc[ss_pos_mask].astype(float)

        if ss_slice.empty:
            tmp[tf1] = pd.Series([np.nan]*len(tf2_list), index=tf2_list, dtype="float64")
            continue

        ranks = ss_slice.rank(method="average")
        s2 = ranks / (len(ss_slice) + 1.0) * 2.0 - 1.0
        s1 = np.abs(s2) * 2.0 - 1.0
        if len(s1) > 0:
            s1 = s1 + (1.0 - float(s1.max())) / 2.0
        s1 = pd.Series(norm.ppf(s1 / 2.0 + 0.5), index=s1.index)

        sign_tf1 = float(np.sign(nes.get(tf1, 0.0))) or 1.0
        s2 = pd.Series(norm.ppf(s2 / 2.0 + 0.5), index=s2.index) * sign_tf1

        # per-pair stat → one-sided p (upper tail)
        per_pair_pvals = []
        for tf2, x in reg_overlap_list:
            x_tfmode = x['tfmode'].astype(float)
            x_like   = x['likelihood'].astype(float)

            if len(x_tfmode) < int(targets):
                per_pair_pvals.append(np.nan)
                continue

            s1_aligned = s1.reindex(x_tfmode.index)
            s2_aligned = s2.reindex(x_tfmode.index)

            sum1 = (x_tfmode * x_like * s2_aligned).sum(skipna=False)
            ss_sign = np.sign(sum1)
            if pd.isna(ss_sign):
                per_pair_pvals.append(np.nan)
                continue
            if ss_sign == 0:
                ss_sign = 1.0

            sum2 = ((1.0 - np.abs(x_tfmode)) * x_like * s1_aligned).sum(skipna=False)
            ww = x_like / x_like.max()
            denom = x_like.sum(skipna=False)
            core = (abs(sum1) + (sum2 if (not pd.isna(sum2) and sum2 > 0) else 0.0)) / denom
            stat = core * ss_sign * np.sqrt((ww**2).sum(skipna=False))

            per_pair_pvals.append(norm.sf(stat) if pd.notna(stat) else np.nan)

        tmp[tf1] = pd.Series(per_pair_pvals, index=tf2_list, dtype="float64")

    # ---- Flatten into a single named p-value vector ("A x B") ----
    flat_vals, flat_names = [], []
    for tf1 in unique_tfs:
        ser = tmp.get(tf1, pd.Series(dtype="float64"))
        tf2_names = list(ser.index)
        flat_vals.extend(ser.values)
        flat_names.extend([f"{tf1} x {tf2}" for tf2 in tf2_names])
    pval_vec = pd.Series(flat_vals, index=pd.Index(flat_names, dtype="object"), dtype="float64")

    # ---- Drop NAs ----
    pval_vec = pval_vec.dropna()

    # ---- Enumerate TF pairs and keep those computed ----
    regind = pd.DataFrame(list(combinations(tfs, 2)), columns=["A", "B"])
    pair_names = regind.apply(lambda r: f"{r['A']} x {r['B']}", axis=1)
    regind = regind[pair_names.isin(pval_vec.index)].reset_index(drop=True)

    # ---- Directional p-value matrix: p(A→B), p(B→A) ----
    ab_names = regind.apply(lambda r: f"{r['A']} x {r['B']}", axis=1)
    ba_names = regind.apply(lambda r: f"{r['B']} x {r['A']}", axis=1)
    pval_mat = pd.DataFrame({
        "A_to_B": pval_vec.reindex(ab_names).values,
        "B_to_A": pval_vec.reindex(ba_names).values
    }, index=regind.index)

    # ---- Count tests per TF (R: table(as.vector(regind))) ----
    tests = pd.Series(regind[["A", "B"]].values.ravel()).value_counts()

    # ==================================================================
    # R 'switch(method, absolute=..., adaptive=...)' — apply penalties
    # ==================================================================
    # Work on a copy so we don't mutate the input regul
    regul_corr: Dict[str, Dict[str, pd.Series]] = {
        tf: {
            "tfmode": (regul[tf]["tfmode"].copy() if "tfmode" in regul[tf] else pd.Series(dtype="float64")),
            "likelihood": (regul[tf]["likelihood"].copy() if "likelihood" in regul[tf] else pd.Series(dtype="float64")),
        }
        for tf in regul
    }
    for tf in regul_corr:
        regul_corr[tf]["tfmode"].index = regul_corr[tf]["tfmode"].index.astype(str)
        regul_corr[tf]["likelihood"].index = regul_corr[tf]["likelihood"].index.astype(str)

    changed: set = set()

    if method == "absolute":
        # Keep pairs where only one direction is significant (<shadow)
        mask_ab = (pval_mat["A_to_B"] < shadow) & (pval_mat["B_to_A"] > shadow)
        mask_ba = (pval_mat["A_to_B"] > shadow) & (pval_mat["B_to_A"] < shadow)
        tmp_abs = pd.concat(
            [
                regind.loc[mask_ab, ["A", "B"]].copy(),
                regind.loc[mask_ba, ["B", "A"]].rename(columns={"B": "A", "A": "B"})
            ],
            ignore_index=True
        )
        if tmp_abs.empty:
            return None

        # Penalize likelihood for the dominated TF (on overlapping targets)
        for _, row in tmp_abs.iterrows():
            dom, oth = row["A"], row["B"]
            if dom not in regul_corr or oth not in regul_corr:
                continue
            dom_tfmode = regul_corr[dom]["tfmode"]
            oth_tfmode = regul_corr[oth]["tfmode"]
            pos_mask = dom_tfmode.index.isin(oth_tfmode.index)

            ll = regul_corr[dom]["likelihood"].copy()
            expo = 1.0 / float(tests.get(dom, 1))  # 1 / tests[dom]
            ll.loc[pos_mask] = ll.loc[pos_mask] / (penalty ** expo)
            regul_corr[dom]["likelihood"] = ll
            changed.add(dom)

    else:  # adaptive
        # Directional evidence = log10(p(B→A)) - log10(p(A→B))
        pval1 = np.log10(pval_mat["B_to_A"].astype(float)) - np.log10(pval_mat["A_to_B"].astype(float))

        # Orient so column 1 is the dominated TF
        pos = pval1 > 0
        neg = pval1 < 0
        tmp_adp = pd.concat(
            [
                regind.loc[pos, ["A", "B"]].copy(),
                regind.loc[neg, ["B", "A"]].rename(columns={"B": "A", "A": "B"})
            ],
            ignore_index=True
        )
        # Magnitude always positive
        pval1_mag = pd.concat([pval1.loc[pos], -pval1.loc[neg]], ignore_index=True)

        if tmp_adp.empty:
            return None

        # Penalize likelihood proportionally to directional evidence
        for i, row in tmp_adp.iterrows():
            dom, oth = row["A"], row["B"]
            if dom not in regul_corr or oth not in regul_corr:
                continue
            w = float(pval1_mag.iloc[i])  # >= 0

            dom_tfmode = regul_corr[dom]["tfmode"]
            oth_tfmode = regul_corr[oth]["tfmode"]
            pos_mask = dom_tfmode.index.isin(oth_tfmode.index)

            ll = regul_corr[dom]["likelihood"].copy()
            expo = penalty / float(tests.get(dom, 1))  # penalty / tests[dom]
            ll.loc[pos_mask] = ll.loc[pos_mask] / ((1.0 + w) ** expo)
            regul_corr[dom]["likelihood"] = ll
            changed.add(dom)

    # ---- Return only those TFs that were actually modified ----
    if not changed:
        return None
    penalized_subset = {tf: regul_corr[tf] for tf in regul if tf in changed}
    return penalized_subset



# Step 8: Prepare the Regulon and NES score to run Pleiotropy 

In [ ]:
import pandas as pd

# 1) Build tt in R/VIPER shape: genes (rows) × samples (cols)
tt = pd.DataFrame(
    gene_expr_signature.X.T,                                   # transpose: AnnData is samples×genes
    index=gene_expr_signature.var_names.astype(str),           # genes
    columns=gene_expr_signature.obs_names.astype(str),         # samples
)

# 2) For the pleiotropy step we’ll extract per-sample columns,
#    but if you want an “ss matrix” like R’s tt, just:
ss = tt.copy()

print(tt.shape, tt.index[:3].tolist(), tt.columns[:3].tolist())

(19938, 1106) ['TSPAN6', 'TNMD', 'DPM1'] ['TCGA-AC-A8OP-01A', 'TCGA-D8-A1XU-01A', 'TCGA-BH-A18L-01A']


In [16]:
# 2) Build NES matrix like R (TFs × samples) from pyviper’s ProtAct_aREA (samples × TFs) 
NES0 = ProtAct_aREA.T.copy() # transpose → rows = TFs, cols = samples 
NES0.index = NES0.index.astype(str) 
NES0.columns = NES0.columns.astype(str)

print("NES0 shape (TFs × samples):", NES0.shape) 
print("TFs (first 3):", list(NES0.index[:3])) 
print("samples (first 3):", list(NES0.columns[:3]))

NES0 shape (TFs × samples): (208, 1106)
TFs (first 3): ['ABLIM3', 'ACTL6B', 'AFF3']
samples (first 3): ['TCGA-AC-A8OP-01A', 'TCGA-D8-A1XU-01A', 'TCGA-BH-A18L-01A']


# Step 9 : Run Pleiotropy 

### What the code does

1. **Loads a benchmark NES table** generated with VIPER (R) after pleiotropy correction  
   → used as ground truth for validation.

2. **Filters the gene expression signature (`ss`)** to the regulon gene universe  
   → reproduces `eset.filter = TRUE` behavior in VIPER.

3. **Implements a VIPER-aligned aREA function (`aREAreg_viper`)**  
   → computes NES using the same ranking, weighting, and normalization strategy as VIPER.

4. **Runs `ShadowRegulon_py` sample-by-sample**  
   → applies pleiotropy penalties to shared targets using the same defaults as R VIPER.

5. **Recomputes NES after pleiotropy correction**  
   → evaluates both *global* and *subset* ranking modes and selects the one closest to R VIPER.

6. **Performs strict benchmark validation**  
   → execution stops if any TF deviates from the R VIPER NES by more than the tolerance (`|ΔNES| > 0.5`).

7. **Updates the NES matrix only after validation passes**  
   → guarantees that stored NES values are fully VIPER-consistent.

### Why this matters

This pipeline ensures that **pyVIPER reproduces pleiotropy-corrected NES values from VIPER (R)**, enabling accurate, reproducible, and fully Python-native transcription factor activity inference.









In [ ]:
from collections import OrderedDict
import pandas as pd
import numpy as np
from typing import Dict, Optional

# --- Load benchmark NES table ---
BENCH_PATH = "/home/friva/SBP_CODE/NES_RVIPER_postcorrectionBRCA1.csv"
TOL = 0.5  # stop if abs diff > 0.5
NES_bench = pd.read_csv(BENCH_PATH, index_col=0)
NES_bench.index = NES_bench.index.astype(str)
NES_bench.columns = NES_bench.columns.astype(str)

# optional progress bar
try:
    from tqdm import tqdm
except ImportError:
    def tqdm(x, **kwargs): return x

# --- one-time integrity checks (same as your single-sample block) ---
s0 = NES0.columns[0]  # just to validate shapes/labels once
assert s0 in ss.columns

tf_in_nes   = set(NES0.index.astype(str))
tf_in_reg   = set(map(str, regulon.keys()))
missing_tfs = tf_in_nes - tf_in_reg
print(f"TFs in NES not in regulon: {len(missing_tfs)}")

all_targets   = set().union(*[set(obj["tfmode"].index.astype(str)) for obj in regulon.values()])
overlap_genes = all_targets & set(ss.index.astype(str))
print(f"Targets in regulon: {len(all_targets)}; overlap with ss: {len(overlap_genes)}")

# --- VIPER parity: filter signature to regulon gene-universe (like eset.filter=TRUE) ---
ss = ss.loc[ss.index.astype(str).isin(all_targets)].copy()

# --- VIPER-aligned aREA (raw likelihoods in sums; normalize only at end) ---
from scipy.special import ndtri

def aREAreg_viper(ss_i: pd.Series,
                  sreg: Dict[str, Dict[str, pd.Series]],
                  minsize: int = 5,
                  mode: str = "global") -> pd.Series:
    """
    VIPER-aligned aREA NES for one sample after pleiotropy.
    mode="global": qnorm(rank/(N+1)) on full gene universe, then subset per TF (canonical aREA).
    mode="subset": qnorm(rank/(n_k+1)) within TF’s subset (shadow-style).
    """
    ss_i = ss_i.astype(float)
    ss_i.index = ss_i.index.astype(str)

    # global transforms
    r_all = ss_i.rank(method="average")
    t2_all = r_all / (len(r_all) + 1.0)                 # (0,1)
    t1_all = (t2_all - 0.5).abs() * 2.0
    t1_all = t1_all + (1.0 - t1_all.max()) / 2.0
    z2_all = pd.Series(ndtri(t2_all), index=ss_i.index)  # qnorm(t2)
    z1_all = pd.Series(ndtri(t1_all), index=ss_i.index)  # qnorm(t1)

    out = {}
    for tf, obj in sreg.items():
        tfmode = pd.Series(obj["tfmode"].values, index=obj["tfmode"].index.astype(str)).astype(float)
        like   = pd.Series(obj["likelihood"].values, index=obj["likelihood"].index.astype(str)).astype(float)

        keep = tfmode.index.intersection(ss_i.index)
        if keep.size < minsize:
            continue

        m = tfmode.loc[keep].values
        l = like.loc[keep].values

        if mode == "global":
            z2 = z2_all.loc[keep].values
            z1 = z1_all.loc[keep].values
        elif mode == "subset":
            sig_sub = ss_i.loc[keep]
            r = sig_sub.rank(method="average")
            t2 = r / (len(r) + 1.0) * 2.0 - 1.0
            t1 = (t2.abs()) * 2.0 - 1.0
            if len(t1) > 0:
                t1 = t1 + (1.0 - float(t1.max())) / 2.0
            z1 = ndtri(t1 / 2.0 + 0.5)
            z2 = ndtri(t2 / 2.0 + 0.5)
            z1, z2 = np.asarray(z1), np.asarray(z2)
        else:
            raise ValueError("mode must be 'global' or 'subset'")

        # guards
        if not np.isfinite(l).any():
            continue
        lmax = np.nanmax(np.abs(l))
        if not np.isfinite(lmax) or lmax == 0:
            continue

        # VIPER formula: raw l in sums; normalize at the end
        sum1 = np.sum(m * l * z2)
        ss_sign = 1.0 if (np.isfinite(sum1) and sum1 >= 0) else -1.0
        sum2 = np.sum((1.0 - np.abs(m)) * l * z1)

        denom = np.sum(l) + 1e-12
        ww = l / lmax
        nes_scale = np.sqrt(np.sum(ww ** 2))

        es_core = (abs(sum1) + (sum2 if (np.isfinite(sum2) and sum2 > 0) else 0.0)) / denom
        out[str(tf)] = float(es_core * ss_sign * nes_scale)

    return pd.Series(out, dtype=float).sort_index()

# --- run ShadowRegulon_py for EVERY sample, then aREAreg to update NES0 ---
sreg_by_sample: Dict[str, Optional[Dict[str, Dict[str, pd.Series]]]] = OrderedDict()
updates_summary: Dict[str, int] = {}

for s in tqdm(list(NES0.columns), desc="shadowRegulon per sample", unit="sample"):
    # build the exact inputs VIPER passes to shadowRegulon()
    ss_i  = ss.loc[:, s]    # Series: genes -> expression for sample s
    nes_i = NES0.loc[:, s]  # Series: TFs   -> uncorrected NES for sample s

    if not isinstance(ss_i, pd.Series):
        ss_i = pd.Series(np.asarray(ss_i).ravel(), index=ss.index.astype(str), name=s)
    if not isinstance(nes_i, pd.Series):
        nes_i = pd.Series(np.asarray(nes_i).ravel(), index=NES0.index.astype(str), name=s)

    # run Python shadowRegulon (VIPER defaults penalty=20, method='adaptive')
    sreg = ShadowRegulon_py(
        ss=ss_i,
        nes=nes_i,
        regul=regulon,
        regulators=0.05,
        shadow=0.05,
        targets=10,
        penalty=20.0,
        method="adaptive",
    )
    sreg_by_sample[s] = sreg

    # print exactly the same style of output as the single-sample test
    if sreg is None:
        print(f"No pleiotropy penalties applied for this sample (sreg=None): {s}")
        updates_summary[s] = 0
        continue
    else:
        print(f"Penalized TFs for sample {s}: {list(sreg.keys())[:5]} ... (total {len(sreg)})")
        tf0 = next(iter(sreg))
        print(tf0, "targets penalized:", len(sreg[tf0]["likelihood"]))

    # --- recompute NES for this sample using VIPER-aligned aREA ---
    nes_g = aREAreg_viper(ss_i, sreg, minsize=5, mode="global")

    # Optional: try 'subset' mode and keep whichever matches the benchmark better
    if s in NES_bench.columns:
        common_tfs_g = nes_g.index.astype(str).intersection(NES_bench.index.astype(str))
        mad_g = (pd.to_numeric(nes_g.loc[common_tfs_g], errors="coerce")
                 - pd.to_numeric(NES_bench.loc[common_tfs_g, s], errors="coerce")).abs().max()

        nes_s = aREAreg_viper(ss_i, sreg, minsize=5, mode="subset")
        common_tfs_s = nes_s.index.astype(str).intersection(NES_bench.index.astype(str))
        mad_s = (pd.to_numeric(nes_s.loc[common_tfs_s], errors="coerce")
                 - pd.to_numeric(NES_bench.loc[common_tfs_s, s], errors="coerce")).abs().max()

        nes_updated = nes_g if (mad_s is np.nan or mad_g <= mad_s) else nes_s
    else:
        nes_updated = nes_g

    # --- compare to benchmark BEFORE writing ---
    if s in NES_bench.columns:
        common_tfs = nes_updated.index.astype(str).intersection(NES_bench.index.astype(str))
        if len(common_tfs) > 0:
            vec_new = pd.to_numeric(nes_updated.loc[common_tfs], errors="coerce")
            vec_bench = pd.to_numeric(NES_bench.loc[common_tfs, s], errors="coerce")
            diffs = (vec_new - vec_bench).abs()

            if (diffs > TOL).any():
                offenders = diffs[diffs > TOL].sort_values(ascending=False)

                # Build a small table with both values + delta
                details = pd.DataFrame({
                    "NES_new":   pd.to_numeric(vec_new.loc[offenders.index], errors="coerce"),
                    "NES_bench": pd.to_numeric(vec_bench.loc[offenders.index], errors="coerce")
                })
                details["Delta_new_minus_bench"] = details["NES_new"] - details["NES_bench"]
                details["AbsDelta"] = details["Delta_new_minus_bench"].abs()
                details = details.sort_values("AbsDelta", ascending=False)

                # Print a compact debug view
                print(f"\n[STOP] Sample '{s}' has {len(offenders)} TFs with |ΔNES| > {TOL}. Top offenders:")
                print(details.head(20).to_string(float_format=lambda x: f"{x:.6f}"))

                # Also save full per-sample diff for offline inspection
                details.to_csv(f"NES_drift_{s}.tsv", sep="\t")

                raise RuntimeError(f"NES deviation exceeds {TOL} for sample {s}. Stopping run.")
        else:
            print(f"[WARN] No common TFs to compare for {s}")
    else:
        print(f"[WARN] Sample {s} not found in benchmark — skipping check")

    # --- only update NES0 if everything passed ---
    common_tfs = nes_updated.index.intersection(NES0.index.astype(str))
    NES0.loc[common_tfs, s] = nes_updated.loc[common_tfs].values
    updates_summary[s] = len(common_tfs)
    print(f"Updated {len(common_tfs)} TFs in sample {s}")


TFs in NES not in regulon: 0
Targets in regulon: 6595; overlap with ss: 6595


shadowRegulon per sample:   0%|                                                                                                        | 1/1106 [00:03<1:09:46,  3.79s/sample]

No pleiotropy penalties applied for this sample (sreg=None): TCGA-AC-A8OP-01A


shadowRegulon per sample:   0%|▏                                                                                                         | 2/1106 [00:06<57:17,  3.11s/sample]

Penalized TFs for sample TCGA-D8-A1XU-01A: ['POU2AF1'] ... (total 1)
POU2AF1 targets penalized: 50
Updated 1 TFs in sample TCGA-D8-A1XU-01A


...

Penalized TFs for sample TCGA-EW-A1IW-01A: ['WNT1'] ... (total 1)
WNT1 targets penalized: 50
Updated 1 TFs in sample TCGA-EW-A1IW-01A


shadowRegulon per sample: 100%|████████████████████████████████████████████████████████████████████████████████████████████████████▉| 1105/1106 [1:17:39<00:03,  3.98s/sample]

No pleiotropy penalties applied for this sample (sreg=None): TCGA-BH-A42V-01A


shadowRegulon per sample: 100%|█████████████████████████████████████████████████████████████████████████████████████████████████████| 1106/1106 [1:17:44<00:00,  4.22s/sample]

No pleiotropy penalties applied for this sample (sreg=None): TCGA-AR-A251-01A


# Steop 10: Save Results for benchmark with R results 

In [18]:
NES0.to_csv("NES_pyVIPER_postcorrectionBRCA1.csv")

In [19]:
NES0

,TCGA-AC-A8OP-01A,TCGA-D8-A1XU-01A,TCGA-BH-A18L-01A,TCGA-B6-A0IK-01A,TCGA-BH-A0B1-01A,TCGA-A8-A06Q-01A,TCGA-D8-A27R-01A,TCGA-E9-A22G-01A,TCGA-AO-A03N-01B,TCGA-E2-A1LE-01A,...,TCGA-BH-A0GY-01A,TCGA-AR-A1AN-01A,TCGA-AC-A23H-01A,TCGA-OL-A5RZ-01A,TCGA-E9-A1RC-01A,TCGA-GM-A2DN-01A,TCGA-B6-A0I1-01A,TCGA-EW-A1IW-01A,TCGA-BH-A42V-01A,TCGA-AR-A251-01A
ABLIM3,3.312995,4.402324,-1.004362,-3.181720,0.155697,-1.816019,-1.988826,-6.355420,1.598219,-1.176965,...,-2.795082,2.713457,-1.950025,-2.324343,1.850295,0.011731,-4.144918,0.850925,4.441164,-5.082763
ACTL6B,0.138986,0.179526,2.151375,0.048878,-0.108404,0.503967,0.268458,0.172664,1.898770,-0.089170,...,0.064317,0.339591,-0.261548,0.729458,-0.133397,0.472961,0.594553,-0.615537,-0.460793,-0.070921
AFF3,1.413288,1.666932,3.103410,-2.066078,0.632171,1.031440,0.576560,-3.476961,-0.257238,-3.497526,...,-1.956222,0.915206,1.165948,-1.591237,0.808338,-1.289979,-4.380195,1.412039,0.829335,-3.337502
AGO2,-1.612328,-2.186245,5.265637,2.308023,2.215284,3.544743,1.199600,4.399653,-2.354983,-1.831523,...,3.430533,-0.599374,4.892562,-0.180102,2.244693,-2.624401,2.769082,1.495621,-7.201144,6.866271
AGRN,3.334114,-2.240458,-4.065179,1.347444,-3.398050,-4.294272,-2.659523,-3.750264,-2.688605,3.078169,...,-3.416673,-0.234432,-2.419282,1.832861,-4.003449,0.360927,-0.436839,-3.623744,6.484625,-5.256523
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
ZMYND15,1.033461,-0.773999,-5.408047,-5.167581,-5.181332,-4.781233,-0.373433,0.433785,0.972662,3.044626,...,-1.395858,-0.923343,-5.750796,-0.529839,-4.142494,5.168600,2.816692,-5.130964,4.134101,-2.013480
ZNF267,-2.721778,2.810309,1.563194,-1.332390,3.391932,3.485368,0.903331,2.289587,3.585383,-2.594329,...,3.503980,0.246713,0.603138,-5.441986,2.712077,3.658692,-0.222014,0.562882,-7.093821,6.637679
ZNF385A,4.275638,2.763715,-0.209267,-0.779405,1.053169,-1.366255,-1.036112,-7.036619,2.617143,0.246459,...,-2.698256,-0.046059,-0.560470,0.219635,0.102116,2.895154,-3.031335,-0.494052,2.909948,-8.083335
ZNF446,4.272559,-1.091221,-0.041467,-4.365264,-1.686188,0.361710,0.171758,-6.117852,0.103318,-0.823877,...,-5.211989,-3.011776,-4.825542,3.554858,0.305532,1.794714,-1.741875,-2.238624,8.023783,-8.767948
